In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from google.colab import files
uploaded = files.upload()

Saving matches_2008-2024.csv to matches_2008-2024 (1).csv


In [ ]:
df = pd.read_csv("matches_2008-2024.csv")
newdf = df.drop(columns="method")
newdf = newdf.dropna()
newdf

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,umpire1,umpire2
0,335982,2008,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,Asad Rauf,RE Koertzen
1,335983,2008,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,MR Benson,SL Shastri
2,335984,2008,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,Aleem Dar,GA Pratapkumar
3,335985,2008,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,SJ Davis,DJ Harper
4,335986,2008,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1426307,2024,Hyderabad,2024-05-19,League,Abhishek Sharma,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Punjab Kings,Sunrisers Hyderabad,Punjab Kings,bat,Sunrisers Hyderabad,wickets,4.0,215.0,20.0,N,Nitin Menon,VK Sharma
1091,1426309,2024,Ahmedabad,2024-05-21,Qualifier 1,MA Starc,"Narendra Modi Stadium, Ahmedabad",Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,8.0,160.0,20.0,N,AK Chaudhary,R Pandit
1092,1426310,2024,Ahmedabad,2024-05-22,Eliminator,R Ashwin,"Narendra Modi Stadium, Ahmedabad",Royal Challengers Bengaluru,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,4.0,173.0,20.0,N,KN Ananthapadmanabhan,MV Saidharshan Kumar
1093,1426311,2024,Chennai,2024-05-24,Qualifier 2,Shahbaz Ahmed,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals,field,Sunrisers Hyderabad,runs,36.0,176.0,20.0,N,Nitin Menon,VK Sharma


In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb


X = newdf[["match_type", "venue", "team1", "team2", "toss_winner", "toss_decision"]]
y = newdf["winner"]

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# One-hot encode input features
X_encoded = pd.get_dummies(X)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.2, random_state=22, stratify=y_encoded
)

# Train the model
model2 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model2.fit(x_train, y_train)

# Save model, feature columns, and label encoder
with open("model2.pkl", "wb") as f:
    pickle.dump(model2, f)

with open("feature_columns.pkl", "wb") as f:
    pickle.dump(x_train.columns.tolist(), f)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:37:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:

import pandas as pd
import pickle
import xgboost as xgb

# --- Load the trained model ---
with open("model2.pkl", "rb") as f:
    model2 = pickle.load(f)

# --- Load the feature columns ---
with open("feature_columns.pkl", "rb") as f:
    feature_columns = pickle.load(f)

# --- Load the label encoder ---
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

ascii_text = r"""__        __   _                            _          _   _
\ \      / /__| | ___ ___  _ __ ___   ___  | |_ ___   | |_| |__   ___
 \ \ /\ / / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \  | __| '_ \ / _ \
  \ V  V /  __/ | (_| (_) | | | | | |  __/ | || (_) | | |_| | | |  __/
 _ \_/\_/_\___|_|\___\___/|_| |_| |_|\___|__\__\___/   \__|_| |_|\___|
| | | | | |_(_)_ __ ___   __ _| |_ ___  |_ _|  _ \| |
| | | | | __| | '_ ` _ \ / _` | __/ _ \  | || |_) | |
| |_| | | |_| | | | | | | (_| | ||  __/  | ||  __/| |___
 \___/|_|\__|_|_| |_| |_|\__,_|\__\___| |___|_|   |_____|
|  _ \ _ __ ___  __| (_) ___| |_ ___  _ __
| |_) | '__/ _ \/ _` | |/ __| __/ _ \| '__|
|  __/| | |  __/ (_| | | (__| || (_) | |
|_|   |_|  \___|\__,_|_|\___|\__\___/|_|

            '.=====.'
            .-\:      /-.
           | (|:.     |) |
            '-|:.     |-'
              \::.    /
               '::. .'
                 ) (
               .' '.
              ⁠ """"""" ⁠

==================================================
|          TEAMS           |        VENUES         |
|--------------------------|-----------------------|
| Chennai Super Kings      | Wankhede Stadium      |
| Mumbai Indians           | Eden Gardens          |
| Kolkata Knight Riders    | M. Chinnaswamy Stadium|
| Royal Challengers Blr    | Arun Jaitley Stadium  |
| Delhi Daredevils (08–18) | Punjab CA Stadium     |
| Delhi Capitals (2019– )  | Sawai Mansingh Stdm   |
| Kings XI Punjab (08–20)  | Rajiv Gandhi Int'l    |
| Punjab Kings (2021– )    | Narendra Modi Stadium |
| Rajasthan Royals         | Ekana Cricket Stadium |
| Sunrisers Hyderabad      | D Y Patil Stadium     |
| Deccan Chargers (08–12)  | Brabourne Stadium     |
| Gujarat Titans (2022– )  | Holkar Cricket Stdm   |
| Gujarat Lions (16–17)    | Green Park            |
| Rising Pune Supergiant(s)| Shaheed Veer Narayan  |
| Pune Warriors India      | MA Chidambaram Stdm   |
| Lucknow Super Giants     | HPCA Stadium          |
|                          | Dr. YS Rajasekhara R  |
|                          | JSCA Stadium          |
|                          | Sheikh Zayed Stadium  |
|                          | Sharjah Cricket Stdm  |
|                          | Dubai Int'l Cricket   |
|                          |                       |
==================================================
"""

print(ascii_text)


# --- Take user input ---
print("\nPlease enter the match details:")

user_input = {
    "match_type": input("Match Type (e.g. League, Eliminator, Final): "),
    "venue": input("Venue: "),
    "team1": input("Team 1 (full team name): "),
    "team2": input("Team 2 (full team name): "),
    "toss_winner": input("Toss Winner (must be team1 or team2): "),
    "toss_decision": input("Toss Decision (bat/field): ")
}

# Validate inputs
if (user_input["team1"] not in le.classes_) or (user_input["team2"] not in le.classes_):
    print("\n❗ One of the teams is not recognized by the model. Please enter exact team names.")
    exit()

if user_input["toss_winner"] not in [user_input["team1"], user_input["team2"]]:
    print("\n❗ Toss winner must be either Team 1 or Team 2.")
    exit()

# --- Convert to DataFrame ---
df_input = pd.DataFrame([user_input])

# --- One-hot encode ---
df_encoded = pd.get_dummies(df_input)

# --- Align with training features (Optimized to prevent fragmentation) ---
df_encoded = df_encoded.reindex(columns=feature_columns, fill_value=0)

# --- Predict probabilities ---
probs = model2.predict_proba(df_encoded)[0]
classes = model2.classes_

# Find the encoded labels
team1_encoded = le.transform([user_input["team1"]])[0]
team2_encoded = le.transform([user_input["team2"]])[0]

# Get indices of teams in prediction output
idx1 = list(classes).index(team1_encoded)
idx2 = list(classes).index(team2_encoded)

# --- Output result ---
print(f'\nIn "{user_input["venue"]}", if "{user_input["toss_winner"]}" won the toss and chose to "{user_input["toss_decision"]}" — The Win probability is:')
print(f'{user_input["team1"]}: {probs[idx1]*100:.2f}%')
print(f'{user_input["team2"]}: {probs[idx2]*100:.2f}%')

guess = input("Based on the probability - Which team do you think will win? : ")
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

# --- Load data and preprocess ---
df = pd.read_csv('cricket_filled_integers.csv')

# Add venue average score
if 'venue' in df.columns:
    venue_avg = df.groupby('venue')['target_runs'].mean().to_dict()
    df['venue_avg_score'] = df['venue'].map(venue_avg)

# Feature engineering
df['is_high_score'] = (df['target_runs'] >= 195).astype(int)
df['is_low_score'] = (df['target_runs'] <= 135).astype(int)

X = df.drop(['target_runs', 'opponent_runs'], axis=1)
y = df['target_runs'].values

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Encoding
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(sparse_output=False), categorical_cols)],
    remainder='passthrough'
)
X_encoded = ct.fit_transform(X)

# Polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=0)

# Train model
xgb = XGBRegressor(n_estimators=150, random_state=0)
xgb.fit(X_train, y_train)

# --- Prediction function ---
def predict_target_score(batting_team, opponent_team, toss_winner, toss_decision, venue):
    user_data = {}
    for col, dtype in zip(X.columns, X.dtypes):
        if col == 'batting_team':
            user_data[col] = [batting_team]
        elif col == 'opponent_team':
            user_data[col] = [opponent_team]
        elif col == 'toss_winner':
            user_data[col] = [toss_winner]
        elif col == 'toss_decision':
            user_data[col] = [toss_decision]
        elif col == 'venue':
            user_data[col] = [venue]
        elif col == 'venue_avg_score':
            venue_avg_dict = df.groupby('venue')['target_runs'].mean().to_dict()
            user_data[col] = [float(venue_avg_dict.get(venue, np.mean(list(venue_avg_dict.values()))))]
        elif col == 'is_high_score' or col == 'is_low_score':
            user_data[col] = [0]
        else:
            if np.issubdtype(dtype, np.number):
                user_data[col] = [0.0]
            else:
                user_data[col] = [X[col].iloc[0]]

    user_df = pd.DataFrame(user_data)

    # Ensure correct types
    for col, dtype in zip(X.columns, X.dtypes):
        if np.issubdtype(dtype, np.number):
            user_df[col] = user_df[col].astype(float)
        else:
            user_df[col] = user_df[col].astype(str)

    user_encoded = ct.transform(user_df)
    user_poly = poly.transform(user_encoded)
    target_pred = xgb.predict(user_poly)
    return int(round(target_pred[0]))


# --- Use previously entered values from the first model input ---
score = predict_target_score(
    batting_team=user_input["team1"],
    opponent_team=user_input["team2"],
    toss_winner=user_input["toss_winner"],
    toss_decision=user_input["toss_decision"],
    venue=user_input["venue"]
)


print("\n🏏 Predicted Target Score for the Match:", score)

df['is_high_opp_score'] = (df['opponent_runs'] >= 195).astype(int)
df['is_low_opp_score'] = (df['opponent_runs'] <= 135).astype(int)

X = df.drop(['target_runs', 'opponent_runs'], axis=1)
y = df['opponent_runs'].values

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(sparse_output=False), categorical_cols)],
    remainder='passthrough'
)

X_encoded = ct.fit_transform(X)

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=0)

xgb = XGBRegressor(n_estimators=150, random_state=0)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
y_pred_xgb_int = np.round(y_pred_xgb).astype(int)

def predict_opponent_score(batting_team, opponent_team, toss_winner, toss_decision, venue):
    venue_avg = df.groupby('venue')['opponent_runs'].mean().to_dict()
    venue_avg_opp_score = venue_avg.get(venue, np.mean(list(venue_avg.values())))

    values = []
    for col in X.columns:
        if col == 'batting_team':
            values.append(batting_team)
        elif col == 'opponent_team':
            values.append(opponent_team)
        elif col == 'toss_winner':
            values.append(toss_winner)
        elif col == 'toss_decision':
            values.append(toss_decision)
        elif col == 'venue':
            values.append(venue)
        elif col == 'venue_avg_opp_score':
            values.append(float(venue_avg_opp_score))
        elif col == 'is_high_opp_score' or col == 'is_low_opp_score':
            values.append(0)
        else:
            val = X[col].iloc[0]
            values.append(val if not pd.isnull(val) else 0)
    user_df = pd.DataFrame([values], columns=X.columns)

    for col, dtype in zip(X.columns, X.dtypes):
        if np.issubdtype(dtype, np.number):
            user_df[col] = user_df[col].astype(float)
        else:
            user_df[col] = user_df[col].astype(str)

    user_encoded = ct.transform(user_df)
    user_poly = poly.transform(user_encoded)
    opp_pred = xgb.predict(user_poly)
    return int(round(opp_pred[0]))

score = predict_opponent_score(
    batting_team=user_input["team1"],
    opponent_team=user_input["team2"],
    toss_winner=user_input["toss_winner"],
    toss_decision=user_input["toss_decision"],
    venue=user_input["venue"]
)

print("Predicted 2nd Innings Score for User Input:", score)

# Assign extracted variables
team1 = user_input["team1"]
team2 = user_input["team2"]
toss_winner = user_input["toss_winner"]
toss_decision = user_input["toss_decision"]
score1 = predict_target_score(
    batting_team=team1,
    opponent_team=team2,
    toss_winner=toss_winner,
    toss_decision=toss_decision,
    venue=user_input["venue"]
)
score2 = predict_opponent_score(
    batting_team=team1,
    opponent_team=team2,
    toss_winner=toss_winner,
    toss_decision=toss_decision,
    venue=user_input["venue"]
)
winner = ""
if toss_winner == team1:
  if toss_decision == 'bat':
    if score1 > score2:
      print(f"{team1} won the match!!")
      winner = team1
    else:
      print(f"{team2} won the match!!")
      winner = team2
  else:
    if score1 > score2:
      print(f"{team2} won the match!!")
      winner = team2
    else:
      print(f"{team1} won the match!!")
      winner = team1
else:
  if toss_decision == 'bat':
    if score1 > score2:
      print(f"{team2} won the match!!")
      winner = team2
    else:
      print(f"{team1} won the match!!")
      winner = team1
  else:
    if score1 > score2:
      print(f"{team1} won the match!!")
      winner = team1
    else:
      print(f"{team2} won the match!!")
      winner = team2
if guess == winner:
  print("Yay! You predicted it right!!!")
else:
  print("Oops, hard luck :)")



__        __   _                            _          _   _
\ \      / /__| | ___ ___  _ __ ___   ___  | |_ ___   | |_| |__   ___
 \ \ /\ / / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \  | __| '_ \ / _ \
  \ V  V /  __/ | (_| (_) | | | | | |  __/ | || (_) | | |_| | | |  __/
 _ \_/\_/_\___|_|\___\___/|_| |_| |_|\___|__\__\___/   \__|_| |_|\___|
| | | | | |_(_)_ __ ___   __ _| |_ ___  |_ _|  _ \| |
| | | | | __| | '_ ` _ \ / _` | __/ _ \  | || |_) | |
| |_| | | |_| | | | | | | (_| | ||  __/  | ||  __/| |___
 \___/|_|\__|_|_| |_| |_|\__,_|\__\___| |___|_|   |_____|
|  _ \ _ __ ___  __| (_) ___| |_ ___  _ __
| |_) | '__/ _ \/ _` | |/ __| __/ _ \| '__|
|  __/| | |  __/ (_| | | (__| || (_) | |
|_|   |_|  \___|\__,_|_|\___|\__\___/|_|

            '.=====.'
            .-\:      /-.
           | (|:.     |) |
            '-|:.     |-'
              \::.    /
               '::. .'
                 ) (
               .' '.
              ⁠ " ⁠

|          TEAMS           |        VENUES         |